In [17]:
import os
from os import kill
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
import time
import pandas as pd
import numpy as np
driver = webdriver.Chrome("C:/chromedriver")  #브라우저 켜기

driver.get('https://pubmed.ncbi.nlm.nih.gov/') # 페이지 이동

# Advanced Search -----------------------------------
toAdvance = driver.find_element_by_css_selector('a.search-input-link')
toAdvance.click()

# title과 abstract으로 설정
title_abstrac = driver.find_element_by_css_selector('select#field-selector')
title_abstrac.click()
opt = driver.find_element_by_css_selector('#field-selector > option:nth-child(39)')
opt.click()

# Input Keywords
search = driver.find_element_by_css_selector('input#id_term')
search.click()
search.send_keys('mimic-iii')
search.send_keys(Keys.ENTER)

option = driver.find_element_by_css_selector('button#add-arrow-button')
option.click()

options = driver.find_elements_by_css_selector('a.boolean-selector-link')
options[1].click()

search.click()
search.send_keys('mimic-iv')
search.send_keys(Keys.ENTER)

searchBtn = driver.find_element_by_css_selector('#search-form > div > div > div.query-box-section-wrapper > div.button-wrapper > button')
searchBtn.click()

# title, Data, Year, Author, Journal, Link

title_arr = []
authors_arr = []
journal_arr = []
link_arr = []
year_arr = []
abstract_arr=[]

# title
titles = driver.find_elements_by_css_selector('a.docsum-title')


# Article Information ---------------------------------------------------------------------------------------
cnt = 0
while True:
    titles = driver.find_elements_by_css_selector('a.docsum-title')
    num = driver.find_element_by_css_selector('#search-results > section > div.search-results-chunks > div > article:nth-child(2) > div.item-selector-wrap.selectors-and-actions.first-selector > label > span').text
    result = driver.find_element_by_css_selector('div.results-amount > span.value').text
    # print("lenght: ", len(titles))
    # print("num:", num)
    if len(titles) <= 10:
        for i in range(len(titles)):
            titles2 = driver.find_elements_by_css_selector('a.docsum-title')
            titles2[i].click()

            #title
            title = driver.find_element_by_css_selector('h1.heading-title')
            title_arr.append(title.text)

            #authors
            authors = driver.find_element_by_css_selector('#full-view-heading > div.inline-authors > div > div > span:nth-child(1) > a')
            authors_arr.append(authors.text)
            
            # journals
            journal = driver.find_element_by_css_selector('#full-view-heading > div.article-citation > div > div > button') 
            journal_arr.append(journal.text)

            # year
            year = driver.find_element_by_css_selector('span.cit')
            year_arr.append(year.text[0:4])

            # abstract
            abstract_all = driver.find_element_by_css_selector('div.abstract-content').text
            if 'MIMIC-III' or '(MIMIC)-III' in abstract_all:
                abstract = 'MIMIC-III'
                abstract_arr.append(abstract)
            elif 'MIMIC-IV' or '(MIMIC)-IV' in abstract_all:
                abstract = 'MIMIC-IV'
                abstract_arr.append(abstract)

            #link
            try:
                link = driver.find_elements_by_css_selector('a.id-link')
                href= link[-1].get_attribute('href')
            except IndexError:
                href = None
            link_arr.append(href)
            
            driver.back()
            cnt += 1
            if i == 9:
                more = driver.find_element_by_css_selector('#search-results > section > div.search-results-paginator.next-results-paginator.has-nav > button > span')
                more.click()
                time.sleep(1)
    
    ## more을 눌러서 20이 된 경우
    elif len(titles) > 10:
        titles2 = driver.find_elements_by_css_selector('a.docsum-title')
        titles2[10].click()
        
        driver.back()

    # Done.
    if cnt == result:
        break


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.abstract-content"}
  (Session info: chrome=91.0.4472.124)


In [4]:
title_arr
authors_arr
journal_arr


['Sci Data',
 'Intensive Care Med',
 'Scand J Trauma Resusc Emerg Med',
 'PLoS One',
 'Intensive Care Med',
 'Intensive Care Med',
 'Lancet Respir Med',
 'Crit Care']

In [115]:
mimic_df['links'].isna().value_counts()

False    219
True     141
Name: links, dtype: int64

In [116]:
mimic_df=pd.DataFrame(np.c_[title_arr, journal_arr, authors_arr, year_arr, link_arr, abstract_arr], columns=['title','journal','authors','years','links','data'])

In [61]:
mimic_df['authors'] = mimic_df['authors'].apply(lambda x: x + " et al.")

In [65]:
import os
os.getcwd()

'c:\\Users\\s_jyhoon94\\crawling\\mimic'

In [78]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(mimic_df[['journal']].value_counts())

journal                                          
J Biomed Inform                                      17
PLoS One                                             16
Sci Rep                                              15
J Am Med Inform Assoc                                15
BMC Med Inform Decis Mak                             14
Biomed Res Int                                       11
Ann Transl Med                                        9
AMIA Jt Summits Transl Sci Proc                       9
Stud Health Technol Inform                            8
Crit Care                                             8
Annu Int Conf IEEE Eng Med Biol Soc                   8
IEEE J Biomed Health Inform                           8
Front Med (Lausanne)                                  7
AMIA Annu Symp Proc                                   6
Comput Biol Med                                       6
Int J Gen Med                                         6
Int J Med Inform                                      

In [117]:
mimic_df['IF']=None

In [118]:
for i in range(mimic_df.shape[0]):
    if mimic_df['journal'][i] == 'J Biomed Inform':
        mimic_df['IF'][i] = 6.317
    elif mimic_df['journal'][i] == "PLoS One":
        mimic_df['IF'][i] = 3.240
    elif mimic_df['journal'][i] == "J Am Med Inform Assoc":
        mimic_df['IF'][i] = 4.112
    elif mimic_df['journal'][i] == "Sci Rep":
        mimic_df['IF'][i] = 3.998
    elif mimic_df['journal'][i] == "BMC Med Inform Decis Mak":
        mimic_df['IF'][i] = 2.067
    elif mimic_df['journal'][i] == "Biomed Res Int":
        mimic_df['IF'][i] = 2.276
    elif mimic_df['journal'][i] == "Ann Transl Med":
        mimic_df['IF'][i] = 3.932
    elif mimic_df['journal'][i] == "Crit Care":
        mimic_df['IF'][i] = 7.598
    elif mimic_df['journal'][i] == "IEEE J Biomed Health Inform":
        mimic_df['IF'][i] = 5.223
    elif mimic_df['journal'][i] == "Comput Biol Med":
        mimic_df['IF'][i] = 3.434
    elif mimic_df['journal'][i] == "Front Med (Lausanne)":
        mimic_df['IF'][i] = 3.9
    elif mimic_df['journal'][i] == "JMIR Med Inform":
        mimic_df['IF'][i] = 2.96
    elif mimic_df['journal'][i] == "Comput Methods Programs Biomed":
        mimic_df['IF'][i] = 3.632
    elif mimic_df['journal'][i] == "Int J Gen Med":
        mimic_df['IF'][i] = 2.45
    elif mimic_df['journal'][i] == "BMJ Open":
        mimic_df['IF'][i] = 2.496
    

In [120]:
mimic_df.to_csv('mimic.csv')

In [5]:
import pandas as pd

In [7]:
mimic=pd.read_csv('mimic.csv')

In [11]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(mimic['journal'].value_counts())

J Biomed Inform                                      17
PLoS One                                             16
J Am Med Inform Assoc                                15
Sci Rep                                              15
BMC Med Inform Decis Mak                             14
Biomed Res Int                                       11
AMIA Jt Summits Transl Sci Proc                       9
Ann Transl Med                                        9
IEEE J Biomed Health Inform                           8
Annu Int Conf IEEE Eng Med Biol Soc                   8
Stud Health Technol Inform                            8
Crit Care                                             8
AMIA Annu Symp Proc                                   6
Int J Gen Med                                         6
Comput Biol Med                                       6
Clin Chim Acta                                        6
Front Med (Lausanne)                                  6
JMIR Med Inform                                 

In [15]:
journal_count = pd.DataFrame(mimic['journal'].value_counts())
journal_count.to_csv('journal_count.csv')

In [12]:
sql="""CASE
  when icd9_code = '39891' then 1
  when icd9_code between '4280' and '4289' then 1
		end as chf       /* Congestive heart failure */

-- cardiac arrhythmias is removed in up to date versions
, case
    when icd9_code = '42610' then 1
    when icd9_code = '42611' then 1
    when icd9_code = '42613' then 1
    when icd9_code between '4262' and '42653' then 1
    when icd9_code between '4266' and '42689' then 1
    when icd9_code = '4270' then 1
    when icd9_code = '4272' then 1
    when icd9_code = '42731' then 1
    when icd9_code = '42760' then 1
    when icd9_code = '4279' then 1
    when icd9_code = '7850' then 1
    when icd9_code between 'V450' and 'V4509' then 1
    when icd9_code between 'V533' and 'V5339' then 1
  end as arythm /* Cardiac arrhythmias */

, CASE
  when icd9_code between '09320' and '09324' then 1
  when icd9_code between '3940' and '3971' then 1
  when icd9_code = '3979' then 1
  when icd9_code between '4240' and '42499' then 1
  when icd9_code between '7463' and '7466' then 1
  when icd9_code = 'V422' then 1
  when icd9_code = 'V433' then 1
		end as valve     /* Valvular disease */

, CASE
  when icd9_code between '41511' and '41519' then 1
  when icd9_code between '4160' and '4169' then 1
  when icd9_code = '4179' then 1
		end as pulmcirc  /* Pulmonary circulation disorder */

, CASE
  when icd9_code between '4400' and '4409' then 1
  when icd9_code between '44100' and '4419' then 1
  when icd9_code between '4420' and '4429' then 1
  when icd9_code between '4431' and '4439' then 1
  when icd9_code between '44421' and '44422' then 1
  when icd9_code = '4471' then 1
  when icd9_code = '449' then 1
  when icd9_code = '5571' then 1
  when icd9_code = '5579' then 1
  when icd9_code = 'V434' then 1
		end as perivasc  /* Peripheral vascular disorder */

, CASE
  when icd9_code = '4011' then 1
  when icd9_code = '4019' then 1
  when icd9_code between '64200' and '64204' then 1
		end as htn       /* Hypertension, uncomplicated */

, CASE
  when icd9_code = '4010' then 1
  when icd9_code = '4372' then 1
		end as htncx     /* Hypertension, complicated */


      /******************************************************************/
      /* The following are special, temporary formats used in the       */
      /* creation of the hypertension complicated comorbidity when      */
      /* overlapping with congestive heart failure or renal failure     */
      /* occurs. These temporary formats are referenced in the program  */
      /* called comoanaly2009.txt.                                      */
      /******************************************************************/
, CASE
  when icd9_code between '64220' and '64224' then 1
		end as htnpreg   /* Pre-existing hypertension complicating pregnancy */

, CASE
  when icd9_code = '40200' then 1
  when icd9_code = '40210' then 1
  when icd9_code = '40290' then 1
  when icd9_code = '40509' then 1
  when icd9_code = '40519' then 1
  when icd9_code = '40599'         then 1
		end as htnwochf  /* Hypertensive heart disease without heart failure */

, CASE
  when icd9_code = '40201' then 1
  when icd9_code = '40211' then 1
  when icd9_code = '40291'         then 1
		end as htnwchf   /* Hypertensive heart disease with heart failure */

, CASE
  when icd9_code = '40300' then 1
  when icd9_code = '40310' then 1
  when icd9_code = '40390' then 1
  when icd9_code = '40501' then 1
  when icd9_code = '40511' then 1
  when icd9_code = '40591' then 1
  when icd9_code between '64210' and '64214' then 1
		end as hrenworf  /* Hypertensive renal disease without renal failure */

, CASE
  when icd9_code = '40301' then 1
  when icd9_code = '40311' then 1
  when icd9_code = '40391'         then 1
		end as hrenwrf   /* Hypertensive renal disease with renal failure */

, CASE
  when icd9_code = '40400' then 1
  when icd9_code = '40410' then 1
  when icd9_code = '40490'         then 1
		end as hhrwohrf  /* Hypertensive heart and renal disease without heart or renal failure */

, CASE
  when icd9_code = '40401' then 1
  when icd9_code = '40411' then 1
  when icd9_code = '40491'         then 1
		end as hhrwchf   /* Hypertensive heart and renal disease with heart failure */

, CASE
  when icd9_code = '40402' then 1
  when icd9_code = '40412' then 1
  when icd9_code = '40492'         then 1
		end as hhrwrf    /* Hypertensive heart and renal disease with renal failure */

, CASE
  when icd9_code = '40403' then 1
  when icd9_code = '40413' then 1
  when icd9_code = '40493'         then 1
		end as hhrwhrf   /* Hypertensive heart and renal disease with heart and renal failure */

, CASE
  when icd9_code between '64270' and '64274' then 1
  when icd9_code between '64290' and '64294' then 1
		end as ohtnpreg  /* Other hypertension in pregnancy */

      /******************** End Temporary Formats ***********************/

, CASE
  when icd9_code between '3420' and '3449' then 1
  when icd9_code between '43820' and '43853' then 1
  when icd9_code = '78072'         then 1
		end as para      /* Paralysis */

, CASE
  when icd9_code between '3300' and '3319' then 1
  when icd9_code = '3320' then 1
  when icd9_code = '3334' then 1
  when icd9_code = '3335' then 1
  when icd9_code = '3337' then 1
  when icd9_code in ('33371','33372','33379','33385','33394') then 1
  when icd9_code between '3340' and '3359' then 1
  when icd9_code = '3380' then 1
  when icd9_code = '340' then 1
  when icd9_code between '3411' and '3419' then 1
  when icd9_code between '34500' and '34511' then 1
  when icd9_code between '3452' and '3453' then 1
  when icd9_code between '34540' and '34591' then 1
  when icd9_code between '34700' and '34701' then 1
  when icd9_code between '34710' and '34711' then 1
  when icd9_code = '3483' then 1 -- discontinued icd-9
  when icd9_code between '64940' and '64944' then 1
  when icd9_code = '7687' then 1
  when icd9_code between '76870' and '76873' then 1
  when icd9_code = '7803' then 1
  when icd9_code = '78031' then 1
  when icd9_code = '78032' then 1
  when icd9_code = '78033' then 1
  when icd9_code = '78039' then 1
  when icd9_code = '78097' then 1
  when icd9_code = '7843'         then 1
		end as neuro     /* Other neurological */

, CASE
  when icd9_code between '490' and '4928' then 1
  when icd9_code between '49300' and '49392' then 1
  when icd9_code between '494' and '4941' then 1
  when icd9_code between '4950' and '505' then 1
  when icd9_code = '5064'         then 1
		end as chrnlung  /* Chronic pulmonary disease */

, CASE
  when icd9_code between '25000' and '25033' then 1
  when icd9_code between '64800' and '64804' then 1
  when icd9_code between '24900' and '24931' then 1
		end as dm        /* Diabetes w/o chronic complications*/

, CASE
  when icd9_code between '25040' and '25093' then 1
  when icd9_code = '7751' then 1
  when icd9_code between '24940' and '24991' then 1
		end as dmcx      /* Diabetes w/ chronic complications */

, CASE
  when icd9_code between '243' and '2442' then 1
  when icd9_code = '2448' then 1
  when icd9_code = '2449'         then 1
		end as hypothy   /* Hypothyroidism */

, CASE
  when icd9_code = '585' then 1 -- discontinued code
  when icd9_code = '5853' then 1
  when icd9_code = '5854' then 1
  when icd9_code = '5855' then 1
  when icd9_code = '5856' then 1
  when icd9_code = '5859' then 1
  when icd9_code = '586' then 1
  when icd9_code = 'V420' then 1
  when icd9_code = 'V451' then 1
  when icd9_code between 'V560' and 'V5632' then 1
  when icd9_code = 'V568' then 1
  when icd9_code between 'V4511' and 'V4512' then 1
		end as renlfail  /* Renal failure */

, CASE
  when icd9_code = '07022' then 1
  when icd9_code = '07023' then 1
  when icd9_code = '07032' then 1
  when icd9_code = '07033' then 1
  when icd9_code = '07044' then 1
  when icd9_code = '07054' then 1
  when icd9_code = '4560' then 1
  when icd9_code = '4561' then 1
  when icd9_code = '45620' then 1
  when icd9_code = '45621' then 1
  when icd9_code = '5710' then 1
  when icd9_code = '5712' then 1
  when icd9_code = '5713' then 1
  when icd9_code between '57140' and '57149' then 1
  when icd9_code = '5715' then 1
  when icd9_code = '5716' then 1
  when icd9_code = '5718' then 1
  when icd9_code = '5719' then 1
  when icd9_code = '5723' then 1
  when icd9_code = '5728' then 1
  when icd9_code = '5735' then 1
  when icd9_code = 'V427'         then 1
		end as liver     /* Liver disease */

, CASE
  when icd9_code = '53141' then 1
  when icd9_code = '53151' then 1
  when icd9_code = '53161' then 1
  when icd9_code = '53170' then 1
  when icd9_code = '53171' then 1
  when icd9_code = '53191' then 1
  when icd9_code = '53241' then 1
  when icd9_code = '53251' then 1
  when icd9_code = '53261' then 1
  when icd9_code = '53270' then 1
  when icd9_code = '53271' then 1
  when icd9_code = '53291' then 1
  when icd9_code = '53341' then 1
  when icd9_code = '53351' then 1
  when icd9_code = '53361' then 1
  when icd9_code = '53370' then 1
  when icd9_code = '53371' then 1
  when icd9_code = '53391' then 1
  when icd9_code = '53441' then 1
  when icd9_code = '53451' then 1
  when icd9_code = '53461' then 1
  when icd9_code = '53470' then 1
  when icd9_code = '53471' then 1
  when icd9_code = '53491'         then 1
		end as ulcer     /* Chronic Peptic ulcer disease (includes bleeding only if obstruction is also present) */

, CASE
  when icd9_code between '042' and '0449' then 1
		end as aids      /* HIV and AIDS */

, CASE
  when icd9_code between '20000' and '20238' then 1
  when icd9_code between '20250' and '20301' then 1
  when icd9_code = '2386' then 1
  when icd9_code = '2733' then 1
  when icd9_code between '20302' and '20382' then 1
		end as lymph     /* Lymphoma */

, CASE
  when icd9_code between '1960' and '1991' then 1
  when icd9_code between '20970' and '20975' then 1
  when icd9_code = '20979' then 1
  when icd9_code = '78951'         then 1
		end as mets      /* Metastatic cancer */

, CASE
  when icd9_code between '1400' and '1729' then 1
  when icd9_code between '1740' and '1759' then 1
  when icd9_code between '179' and '1958' then 1
  when icd9_code between '20900' and '20924' then 1
  when icd9_code between '20925' and '2093' then 1
  when icd9_code between '20930' and '20936' then 1
  when icd9_code between '25801' and '25803' then 1
		end as tumor     /* Solid tumor without metastasis */

, CASE
  when icd9_code = '7010' then 1
  when icd9_code between '7100' and '7109' then 1
  when icd9_code between '7140' and '7149' then 1
  when icd9_code between '7200' and '7209' then 1
  when icd9_code = '725' then 1
		end as arth              /* Rheumatoid arthritis/collagen vascular diseases */

, CASE
  when icd9_code between '2860' and '2869' then 1
  when icd9_code = '2871' then 1
  when icd9_code between '2873' and '2875' then 1
  when icd9_code between '64930' and '64934' then 1
  when icd9_code = '28984'         then 1
		end as coag      /* Coagulation deficiency */

, CASE
  when icd9_code = '2780' then 1
  when icd9_code = '27800' then 1
  when icd9_code = '27801' then 1
  when icd9_code = '27803' then 1
  when icd9_code between '64910' and '64914' then 1
  when icd9_code between 'V8530' and 'V8539' then 1
  when icd9_code = 'V854' then 1 -- hierarchy used for AHRQ v3.6 and earlier
  when icd9_code between 'V8541' and 'V8545' then 1
  when icd9_code = 'V8554' then 1
  when icd9_code = '79391'         then 1
		end as obese     /* Obesity      */

, CASE
  when icd9_code between '260' and '2639' then 1
  when icd9_code between '78321' and '78322' then 1
		end as wghtloss  /* Weight loss */

, CASE
  when icd9_code between '2760' and '2769' then 1
		end as lytes     /* Fluid and electrolyte disorders - note:
                                      this comorbidity should be dropped when
                                      used with the AHRQ Patient Safety Indicators*/
, CASE
  when icd9_code = '2800' then 1
  when icd9_code between '64820' and '64824' then 1
		end as bldloss   /* Blood loss anemia */

, CASE
  when icd9_code between '2801' and '2819' then 1
  when icd9_code between '28521' and '28529' then 1
  when icd9_code = '2859'         then 1
		end as anemdef  /* Deficiency anemias */

, CASE
  when icd9_code between '2910' and '2913' then 1
  when icd9_code = '2915' then 1
  when icd9_code = '2918' then 1
  when icd9_code = '29181' then 1
  when icd9_code = '29182' then 1
  when icd9_code = '29189' then 1
  when icd9_code = '2919' then 1
  when icd9_code between '30300' and '30393' then 1
  when icd9_code between '30500' and '30503' then 1
		end as alcohol   /* Alcohol abuse */

, CASE
  when icd9_code = '2920' then 1
  when icd9_code between '29282' and '29289' then 1
  when icd9_code = '2929' then 1
  when icd9_code between '30400' and '30493' then 1
  when icd9_code between '30520' and '30593' then 1
  when icd9_code between '64830' and '64834' then 1
		end as drug      /* Drug abuse */

, CASE
  when icd9_code between '29500' and '2989' then 1
  when icd9_code = '29910' then 1
  when icd9_code = '29911'         then 1
		end as psych    /* Psychoses */

, CASE
  when icd9_code = '3004' then 1
  when icd9_code = '30112' then 1
  when icd9_code = '3090' then 1
  when icd9_code = '3091' then 1
  when icd9_code = '311'         then 1"""

In [13]:
import re
p2=re.compile("\w{4} \w{3}\d{1}_\w{4} = '\d{3,5}'")
p2.findall(sql)

["when icd9_code = '39891'",
 "when icd9_code = '42610'",
 "when icd9_code = '42611'",
 "when icd9_code = '42613'",
 "when icd9_code = '4270'",
 "when icd9_code = '4272'",
 "when icd9_code = '42731'",
 "when icd9_code = '42760'",
 "when icd9_code = '4279'",
 "when icd9_code = '7850'",
 "when icd9_code = '3979'",
 "when icd9_code = '4179'",
 "when icd9_code = '4471'",
 "when icd9_code = '449'",
 "when icd9_code = '5571'",
 "when icd9_code = '5579'",
 "when icd9_code = '4011'",
 "when icd9_code = '4019'",
 "when icd9_code = '4010'",
 "when icd9_code = '4372'",
 "when icd9_code = '40200'",
 "when icd9_code = '40210'",
 "when icd9_code = '40290'",
 "when icd9_code = '40509'",
 "when icd9_code = '40519'",
 "when icd9_code = '40599'",
 "when icd9_code = '40201'",
 "when icd9_code = '40211'",
 "when icd9_code = '40291'",
 "when icd9_code = '40300'",
 "when icd9_code = '40310'",
 "when icd9_code = '40390'",
 "when icd9_code = '40501'",
 "when icd9_code = '40511'",
 "when icd9_code = '40591'",


In [14]:
import re
p3 = re.compile("\w{4} \w{3}\d{1}_\w{4} \w{7} '\d{3,5}' \w{3} '\d{3,5}'|\w{4} \w{3}\d{1}_\w{4} \w{7} '\w{1}\d{3,5}' \w{3} '\w{1}\d{3,5}'")
p3.findall(sql)

["when icd9_code between '4280' and '4289'",
 "when icd9_code between '4262' and '42653'",
 "when icd9_code between '4266' and '42689'",
 "when icd9_code between 'V450' and 'V4509'",
 "when icd9_code between 'V533' and 'V5339'",
 "when icd9_code between '09320' and '09324'",
 "when icd9_code between '3940' and '3971'",
 "when icd9_code between '4240' and '42499'",
 "when icd9_code between '7463' and '7466'",
 "when icd9_code between '41511' and '41519'",
 "when icd9_code between '4160' and '4169'",
 "when icd9_code between '4400' and '4409'",
 "when icd9_code between '44100' and '4419'",
 "when icd9_code between '4420' and '4429'",
 "when icd9_code between '4431' and '4439'",
 "when icd9_code between '44421' and '44422'",
 "when icd9_code between '64200' and '64204'",
 "when icd9_code between '64220' and '64224'",
 "when icd9_code between '64210' and '64214'",
 "when icd9_code between '64270' and '64274'",
 "when icd9_code between '64290' and '64294'",
 "when icd9_code between '3420' a

In [2]:
import pandas as pd
import numpy as np
import re

p = re.compile('\d{3,5}|\w{1}\d{3}')

p.findall(sql)

['42610',
 '42611',
 '42613',
 '4262',
 '42653',
 '4266',
 '42689',
 '4270',
 '4272',
 '42731',
 '42760',
 '4279',
 '7850',
 'V450',
 'V450',
 'V533',
 'V533']